In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/philippe/MolGenDocking


/home/philippe/miniconda3/envs/MolGen/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import numpy as np
import json
import MDAnalysis as mda
import nglview as nv

In [68]:
PATH = 'data/SIU/pdb_files'

pdb_ids = [ids.replace(".pdb", "") for ids in os.listdir(PATH)]
len(pdb_ids)

6354

In [69]:
i=0

In [70]:
def get_selection(size, center, sim):
    dx, dy, dz = size
    x, y, z = center

    # Compute min/max box edges
    xmin, xmax = x - dx/2, x + dx/2
    ymin, ymax = y - dy/2, y + dy/2
    zmin, zmax = z - dz/2, z + dz/2

    # Get atoms
    all_atoms = sim.select_atoms("all")
    coords = all_atoms.positions

    # Create boolean mask for atoms inside the box
    mask = (
        (coords[:, 0] > xmin) & (coords[:, 0] < xmax) &
        (coords[:, 1] > ymin) & (coords[:, 1] < ymax) &
        (coords[:, 2] > zmin) & (coords[:, 2] < zmax)
    )

    # Apply mask
    pocket_atoms = all_atoms[mask]
    pocket_residues = pocket_atoms.residues

    # Use segid instead of chainID
    pocket_selection = " or ".join([f"resi {res.resid}" for res in pocket_residues])
    return pocket_selection


In [78]:
# Write txt files for fpocket
files = [os.path.join(PATH, pdb_id + ".pdb") for pdb_id in pdb_ids]
BATCH_SIZE = 16

for idx in range(0, len(files) // BATCH_SIZE + len(files) % BATCH_SIZE):
    idx_min = idx * BATCH_SIZE
    idx_max = min((idx + 1) * BATCH_SIZE, len(files))
    print(f"Processing batch {idx+1}/{len(files) // BATCH_SIZE + 1}, files {idx_min+1}-{idx_max}")
    batch_files = files[idx_min:idx_max]
    # Create directory if it doesn't exist
    os.makedirs("data/SIU/batch_fpockets", exist_ok=True)
    with open(f"data/SIU/batch_fpockets/batch_{idx+1}.txt", "w") as f:
        for file in batch_files:
            f.write(file + "\n")

Processing batch 1/398, files 1-16
Processing batch 2/398, files 17-32
Processing batch 3/398, files 33-48
Processing batch 4/398, files 49-64
Processing batch 5/398, files 65-80
Processing batch 6/398, files 81-96
Processing batch 7/398, files 97-112
Processing batch 8/398, files 113-128
Processing batch 9/398, files 129-144
Processing batch 10/398, files 145-160
Processing batch 11/398, files 161-176
Processing batch 12/398, files 177-192
Processing batch 13/398, files 193-208
Processing batch 14/398, files 209-224
Processing batch 15/398, files 225-240
Processing batch 16/398, files 241-256
Processing batch 17/398, files 257-272
Processing batch 18/398, files 273-288
Processing batch 19/398, files 289-304
Processing batch 20/398, files 305-320
Processing batch 21/398, files 321-336
Processing batch 22/398, files 337-352
Processing batch 23/398, files 353-368
Processing batch 24/398, files 369-384
Processing batch 25/398, files 385-400
Processing batch 26/398, files 401-416
Processin

In [75]:
len(files)

6354

In [72]:
pdb_id = pdb_ids[i]
i+=1
print(pdb_id)
u = mda.Universe(os.path.join(PATH, f"{pdb_id}.pdb"))
#
# center = pockets[pdb_id]['center']
# radius = np.mean(size)/10
#
# pocket_selection = get_selection(
#     size=np.array(pockets[pdb_id]['size']),
#     center=np.array(center),
#     sim=u
# )

view = nv.show_mdanalysis(u)

view.add_representation("surface", colorScheme="hydrophobicity")
view

5ijc


NGLWidget()

In [51]:
x

52.434799999999996